# Dataflow for Storage Optimization
## Inputs
The code will recieve a signal from D365. Based on the contents of that message, the logic should either decide to simply output D365's message without overriding anything or modify D365's message with the appropriate storage location.

## Outputs
The code will output the signal from D365 either unmodified (if the drop name == 'stage') or with the storage optimization's output incorporated into the signal.

## Transactional History
We are required to keep a history of recent decisions made by storage optimization - e.g., if we send a container to a location, we need to make sure we don't immediately send another one there. A simple JSON should be sufficient. We can override the JSON over X time interval.

## Current Warehouse State
A current state of storage can be determined using the D365 database to identify available storage space. 
To identify **unavailable** space, we can use a query like so:
1. Select LICENSEPLATENUMBER and WAREHOUSELOCATIONID  from the inventory on-hand table  
2. Left join with the license plates table on LICENCEPLATENUMBER and select CONTAINERTYPEID  

In [9]:
import json
import os
import sqlite3
from datetime import datetime
import pandas as pd
import pyodbc

### Create Transaction History DB + Set Index
Set LoadIdentifier as the index so we can update values in the future instead of adding duplicate rows

In [4]:
# create transaction history db
db_path = './thistory.db' # change path as needed

cols = {
    'LoadIdentifier':'TEXT', # license plate number
    'AssignmentTime':'TEXT', # last time our code touched it
    'StagingLoc':'TEXT', # staging location
    'FinalLoc':'TEXT', # final location
}

# create main table with columns
query = 'CREATE TABLE IF NOT EXISTS TransactionHistory ('
for col, ctype in cols.items():
    col = "\"{}\"".format(col)
    query = query + col + ' ' + ctype + ', '
query = query.strip(', ') + ')'
connection = sqlite3.connect(db_path)
cursor = connection.cursor()
# create table
cursor.execute(query)
connection.commit()

# set index
# cursor = connection.cursor()
# query = '''
#     CREATE UNIQUE INDEX "LoadIdx" 
#     ON TransactionHistory (LoadIdentifier);
# '''
# cursor.execute(query)
# connection.commit()

In [1]:

import numpy as np
import os
from pprint import pprint
import pandas as pd
import pulp
import random
# import storage
from storage.warehouse_input_parser import WarehouseInputParser
from storage.containers import Bin, Tote, BulkContainer
from storage.item import Item
import time





    
def get_units(n_units=3):
    # create Item objects
    # if N_UNITS is None:
    #     N_UNITS=
    container_types = (Bin, Tote, BulkContainer)
    containers = []
    items = []


    for i in range(n_units):
        container = np.random.choice(container_types, p=[0.8,0.1,0.1])()
        container.priority = np.random.choice(('A','B','C'), p=[0.1,0.8,0.1])
        
        # btw 1 and 6 items per container
        n_items = random.randint(1,6)
        for i in range(n_items):
            item = Item(priority = container.priority)
            items.append(item)
            c = container
            c.d = c.dimensions['outer']
            c.l, c.w, c.h = c.d['length'], c.d['width'], c.d['height']
            c.add_item_to_container(item)
            

        containers.append(c)

        
    units = [ 
            {'ID':c.uuid.hex,
             'Name': c.name,
             'Full Name': c.full_name,
             'Length':c.dimensions['outer']['length'], 
             'Width':c.dimensions['outer']['width'], 
             'Height':c.dimensions['outer']['height'],
             'Volume': c.l*c.w*c.h,
             'Priority':c.priority,
             'Weight': min(c.weight, 2000)} for c in containers ]
    
    # for unit in units:
    #     unit['Volume'] = unit['Length']*unit['Width']*unit['Height']


    return items, containers, units




# return the unit by ID
def get_unit_by_id(unitID):
    for unit in units:
        if unitID == unit['ID']:
            return unit

# return the item by ID
def get_item_by_id(unitID):
    for item in items:
        if unitID == item.uuid:
            return item

def get_container_by_id(containerID):
    for container in containers:
        if containerID == container.uuid.hex:
            return container
    

def get_warehouse(warehouse_path='warehouse-layout-input.xlsx'):
    
    warehouse = WarehouseInputParser(warehouse_path).process_warehouse_input()
    all_shelf_ids = list(warehouse['ShelfID'])
    shelves = warehouse.to_dict('records')
    for shelf in shelves:
        shelf['Volume'] = shelf.get('ShelfLength')*shelf.get('ShelfWidth')*shelf.get('ShelfHeight')
    shelf_count = len(all_shelf_ids)
    
    return warehouse, all_shelf_ids, shelves, shelf_count

# warehouse, all_shelf_ids, shelves, shelf_count = get_warehouse()

def get_shelf_by_id(ShelfID):
    for shelf in shelves:
        if shelf['ShelfID'] == ShelfID:
            return shelf
        

# items, containers, units = get_units(n_units=N_UNITS)



def optimize_function(  x=dict(), # old soln x
                        # y=dict(), # old soln y
                        n_units=3, # number of units to add
                        old_units=[],
                        new_units=[],
                        shelves=None,
                        all_shelf_ids=None):


    # store solution for x
    solutionX = dict()
    for k, v in x.items():
        solutionX[k] = v.value()

    # # store solution for y
    # solutionY = dict()
    # for k, v in y.items():
    #     solutionY[k] = v.value()

    units = old_units + new_units

    print(f'There are now {len(units)} units.')




    '''
    Variables
    '''

    shelf_loc = [(u.get('ID'), shelfID) for u in units for shelfID in all_shelf_ids]


    # # x is options for item + bin combinations
    x = pulp.LpVariable.dicts(
        'OnShelf', shelf_loc,
        lowBound = 0, # item is not on the shelf
        upBound = 1, # item is on the shelf
        cat = 'Integer' # solver uses integers, i think?
    )

    # z is a dummy variable for shelf waste; minimizing it then using it in a constraint acts like taking the max would
    z = pulp.LpVariable.dicts(
        'ShelfWaste', [(unit.get('ID'), shelf.get('ShelfID')) for unit in units for shelf in shelves],
        0,
        cat = 'Integer'
    )

    # add bounds unique to each shelf
    for i in z:
        for shelf in shelves:
            if shelf.get('ShelfID') == i:
                z[i].bounds(0, shelf.get('Volume'))


    # initialize problem
    prob = pulp.LpProblem('Shelf_Stocking', pulp.LpMinimize)

    '''
    Objective Function
    '''
    prob += pulp.lpSum([z[(unit.get('ID'), shelf.get('ShelfID'))] for unit in units for shelf in shelves]), 'Objective: Minimize Excess Shelf Space'

    '''
    constraints
    '''

    # enforce old solution
    for k,v in solutionX.items():
        for i in range(shelf_count):
            if int(v) == 1:
                try:
                    prob += x[k] == 1.0, f'Enforce old solution {k}'
                except:
                    pass

    for unit in units:
        prob += pulp.lpSum([x[(unit.get('ID'), shelfID)] for shelfID in all_shelf_ids]) == 1, 'Item {} can only be on one shelf'.format(unit.get('ID'))


    # # sum of item widths should not exceed shelf width ("length")
    for i, shelf in enumerate(shelves):
        prob += pulp.lpSum([unit.get('Length') * x[(unit.get('ID'), shelf['ShelfID'])] for unit in units]) <= shelf['ShelfLength'], 'Sum of widths cannot exceed shelf {} width'.format(i)


        # the sum of item weights should not exceed shelf capacity
        prob += pulp.lpSum([unit.get('Weight') * x[(unit.get('ID'), shelf['ShelfID'])] for unit in units]) <= shelf['WeightCapacity'], 'Sum of weights cannot exceed shelf {} weight cap'.format(i)


        for unit in units:
            # minimize shelf waste
            prob += ( shelf.get('Volume') - unit.get('Volume')) * x[(unit.get('ID'), shelf.get('ShelfID'))] <= z[unit.get('ID'), shelf.get('ShelfID')], 'Minimize wasted space created by inserting item item {} onto shelf {}'.format(unit.get('ID'), shelf.get('ShelfID'))


    # the difference in priority btw item and shelf should be 0
    prob+= pulp.lpSum([abs(ord(unit.get('Priority')) - ord(shelf.get('ShelfPriority'))) * x[(unit.get('ID'), shelf['ShelfID'])]\
                        for unit in units for shelf in shelves ]) == 0, 'All items are matched to priority'
    
    
    # set initial values
    for k, v in solutionX.items():
        if int(v) == 1:
            # print(k,'-->',v)
            x[k].setInitialValue(v)
    
    

    # '''
    # Solve
    # '''
    start_time = time.time()
    solver = pulp.PULP_CBC_CMD(msg=True)
    prob.solve(solver)
    print('Solved in {:.3f} seconds.'.format(time.time() - start_time))

    return x, z, items, containers, units


def get_stocked_shelves(x):
    stocked_shelves = {}
    for item in x.keys():
        if x[item].value()==1:
            itemNum = item[0]
            shelfNum = item[1]
            if shelfNum in stocked_shelves:
                stocked_shelves[shelfNum].append(itemNum)
            else:
                stocked_shelves[shelfNum] = [itemNum]
    return stocked_shelves

# pprint(get_stocked_shelves(x))



def detail_report(stocked_shelves, shelves, units):
 
    for shelfID, unitIDs in stocked_shelves.items():
        print(f'\nShelf {shelfID}:')
        shelfdata = get_shelf_by_id(shelfID, shelves)
        print(f"\t{shelfdata['ShelfType']}\tpriotiry: {shelfdata['ShelfPriority']}\tcontains {unitIDs}\n")

 
        for unitID in unitIDs:
            unit = get_unit_by_id(unitID, units)
            pprint(unit)


def put_away(units=[],
             x=dict(),
             used_shelf_ids=[],
             warehouse_path='warehouse-layout-input.xlsx'):

    warehouse, all_shelf_ids, shelves, shelf_count = get_warehouse(warehouse_path=warehouse_path)

    all_shelf_ids = list(set(all_shelf_ids)-set(used_shelf_ids))
    shelves = [ shelf for shelf in shelves if shelf['ShelfID'] in all_shelf_ids ]
    shelf_count = len(shelves)
    
    xold = x
    x, z, items, containers, units = optimize_function(x=xold,
                                                       new_units=units,
                                                      all_shelf_ids=all_shelf_ids,
                                                      shelves=shelves)

    return get_stocked_shelves(x)

def storage_opt(signal, dbcon=None):
    action = [s for s in signal['Locations'] if s['Operation']=='drop'][0]['Name']
    # if signal loc for 'drop' is named 'stage', return signal as-is
    try:
        if action == 'Stage':
            # we will modify this later
            return signal
        elif action == 'Final':
            
            # parse additional data as needed
            data = signal['KPMGAdditionalData']
            # container type
            container_type = data['ContainerType']
            # container priority
            container_priority = data['priority']
            # container dimensions - useful for pallets with unknown heights
            container_height = data['height']
            container_width = data['width']
            container_depth = data['depth']
            container_weught = data['weight']
            
            container_id = signal['LoadIdentifier']
            
            unit = [{'ID': container_id,
                      'Name': container_type,
                      'Length': container_width,
                      'Width': container_depth,
                      'Height': container_height,
                      'Volume': container_width*container_depth*container_height,
                      'Priority': container_priority,
                      'Weight': container_weight}]
            
            '''
            INSERT CODE TO DETERMINE CURRENT STORAGE AVAILABILITY (AZURE SQL)
            '''
    

            if dbcon is None:
                print('Establishing connection to inventory DB')
                dbcon = pyodbc.connect('''Driver={ODBC Driver 18 for SQL Server};Server=tcp:usmc5gsmartwarehousetest.database.windows.net,1433;Database=D365TestDB;Uid=usmc5gadmin;Pwd=Usmc5gp@ssword1;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;''')



            # cursor = connection.cursor()
            query = '''
                SELECT
                s.LICENSEPLATENUMBER AS "LPN",
                l.LICENSEPLATENUMBER AS "LPN2",
                l.CONTAINERTYPEID AS "ContainerType",
                l.PARENTLICENSEPLATENUMBER as "ParentLPN",
                p.CONTAINERTYPEID AS "ParentContainerType",
                s.WAREHOUSELOCATIONID
                FROM "Inventory On-Hand Report Storage" s
                LEFT JOIN "License Plates" l on s.LICENSEPLATENUMBER = l.LICENSEPLATENUMBER
                LEFT JOIN "License Plates" p on l.PARENTLICENSEPLATENUMBER = p.LICENSEPLATENUMBER
                '''
            # cursor.execute(query)
            # column_names = [x[0] for x in cursor.description]
            # extract = cursor.fetchall()
            # df = pd.DataFrame(extract, columns=column_names)

            # connection.close()
            df = pd.read_sql(query, dbcon)

            df = df[df['WAREHOUSELOCATIONID'].notna()]
            df = df[df['WAREHOUSELOCATIONID'].str.match('.2.\d{4}\w{2}')]
            df = df.drop(columns=['LPN2']) # duplicate
            # if an item has a parent lpn, use that instead
            df.loc[df['ParentLPN'].notna(), 'LPN'] = df['ParentLPN']
            # if an item has a parent lpn, use that container type instead
            df.loc[df['ParentContainerType'].notna(), 'ContainerType'] = df['ParentContainerType']
            # drop parent cols now
            df = df.drop(columns=['ParentLPN', 'ParentContainerType'])
            # drop duplicates created from multiple rows having the same parent info
            df = df[~df.duplicated()]
            
            container_ids = [
                'MODULAR NESTING CONT',
                'BULK CONT LENGTHDOOR',
                'BULK CONT WIDTHDOOR',
                'WOOD SHIPPING PALLET'
            ]

            # df
            def loc_id_to_tuple(locid):
                l = locid
                return (int(l[3:5]), int(l[5:7]),l[7])

            mod_nest_counter = {loc_id_to_tuple(locid):0 for locid in df.WAREHOUSELOCATIONID}
            MOD_NEST_MAX = 3
            used_shelf_ids = []
            for idx, row in df.iterrows():
                 # df
                locid = loc_id_to_tuple(row.WAREHOUSELOCATIONID)
                
                if row.ContainerType == 'MODULAR NESTING CONT':
                    mod_nest_counter[locid]+=1
                elif row.ContainerType in ['BULK CONT LENGTHDOOR','BULK CONT WIDTHDOOR','WOOD SHIPPING PALLET']:
                    used_shelf_ids.append(locid)
                    
            for locid, num_mod_nest in mod_nest_counter.items():
                if num_mod_nest > MOD_NEST_MAX:
                    used_shelf_ids.append(locid)
                    
            
            
                
            
            
            
            '''
            READ TRANSACTION HISTORY + UPDATE STORAGE AVAILABILITY
            '''
            # get relevant transaction history data
            connection = sqlite3.connect(db_path)
            cursor = connection.cursor()
            query = '''SELECT * FROM TransactionHistory'''
            cursor.execute(query)
            column_names = [x[0] for x in cursor.description]
            extract = cursor.fetchall()
            df = pd.DataFrame(extract, columns=column_names)
            connection.close()
            
            
            # update storage availability with any updates that we've processed in the last 30 min
            df['AssignmentTime'] = pd.to_datetime(df['AssignmentTime'])
            
            
            '''
            INSERT STORAGE OPTIMIZATION CODE WHERE OUTPUT == final_location
            '''
            
            new = put_away(units=unit,
                               used_shelf_ids=used_shelf_ids)
            new = list(new.keys())[0]
            
            def to_loc_fmt(key_fmt):
                return 'I21'+''.join(list(key_fmt))+'A'
            
            
            final_location = to_loc_fmt(new) # just a sample output that storage may have, but output must be in this format
            
            
            '''
            WRITE DECISION TO TRANSACTION HISTORY
            '''
            connection = sqlite3.connect(db_path)
            cursor = connection.cursor()
            query = '''
                INSERT OR REPLACE INTO TransactionHistory ("LoadIdentifier", "AssignmentTime", "FinalLoc")
                VALUES (?, ?, ?)
            '''
            cursor.execute(query, [signal['LoadIdentifier'], datetime.now().strftime('%Y-%m-%d %H:%M:%S'), final_location])
            connection.commit()
            connection.close()
                    
            # drop KPMGAdditionalData from output - TMO doesn't need it
            signal.pop('KPMGAdditionalData', None)
                    
            return signal 
        else:
            raise Exception('Input data not "final" or "stage".')
    except:
        raise Exception('Error parsing input data.')

In [2]:
warehouse_path='warehouse-layout-input.xlsx'
warehouse, all_shelf_ids, shelves, shelf_count = get_warehouse(warehouse_path=warehouse_path)
shelves

[{'ShelfBay': 2,
  'ShelfType': 'Standard',
  'ShelfColumn': 51,
  'ShelfAisle': 3,
  'ShelfPriority': 'C',
  'AdjustableConfig': nan,
  'Compartment': 'A',
  'ShelfID': (51, 3, 'A'),
  'ShelfLength': 54.0,
  'ShelfWidth': 42.0,
  'ShelfHeight': 60.0,
  'WeightCapacity': 2530.0,
  'Volume': 136080.0},
 {'ShelfBay': 2,
  'ShelfType': 'Standard',
  'ShelfColumn': 51,
  'ShelfAisle': 3,
  'ShelfPriority': 'C',
  'AdjustableConfig': nan,
  'Compartment': 'B',
  'ShelfID': (51, 3, 'B'),
  'ShelfLength': 54.0,
  'ShelfWidth': 42.0,
  'ShelfHeight': 60.0,
  'WeightCapacity': 2530.0,
  'Volume': 136080.0},
 {'ShelfBay': 2,
  'ShelfType': 'Standard',
  'ShelfColumn': 51,
  'ShelfAisle': 3,
  'ShelfPriority': 'C',
  'AdjustableConfig': nan,
  'Compartment': 'C',
  'ShelfID': (51, 3, 'C'),
  'ShelfLength': 54.0,
  'ShelfWidth': 42.0,
  'ShelfHeight': 60.0,
  'WeightCapacity': 2530.0,
  'Volume': 136080.0},
 {'ShelfBay': 2,
  'ShelfType': 'Standard',
  'ShelfColumn': 51,
  'ShelfAisle': 4,
  'Shel

In [4]:
all_shelf_ids

[(51, 3, 'A'),
 (51, 3, 'B'),
 (51, 3, 'C'),
 (51, 4, 'A'),
 (51, 4, 'B'),
 (51, 4, 'C'),
 (51, 6, 'A'),
 (51, 6, 'B'),
 (51, 6, 'C'),
 (51, 7, 'A'),
 (51, 7, 'B'),
 (51, 7, 'C'),
 (51, 9, 'A'),
 (51, 9, 'B'),
 (51, 9, 'C'),
 (51, 10, 'A'),
 (51, 10, 'B'),
 (51, 10, 'C'),
 (51, 17, 'A'),
 (51, 17, 'B'),
 (51, 17, 'C'),
 (51, 18, 'A'),
 (51, 18, 'B'),
 (51, 18, 'C'),
 (51, 20, 'A'),
 (51, 20, 'B'),
 (51, 20, 'C'),
 (51, 21, 'A'),
 (51, 21, 'B'),
 (51, 21, 'C'),
 (51, 23, 'A'),
 (51, 23, 'B'),
 (51, 23, 'C'),
 (51, 24, 'A'),
 (51, 24, 'B'),
 (51, 24, 'C'),
 (51, 26, 'A'),
 (51, 26, 'B'),
 (51, 26, 'C'),
 (51, 27, 'A'),
 (51, 27, 'B'),
 (51, 27, 'C'),
 (51, 29, 'A'),
 (51, 29, 'B'),
 (51, 29, 'C'),
 (51, 34, 'A'),
 (51, 34, 'B'),
 (51, 34, 'C'),
 (51, 35, 'A'),
 (51, 35, 'B'),
 (51, 35, 'C'),
 (51, 37, 'A'),
 (51, 37, 'B'),
 (51, 37, 'C'),
 (51, 38, 'A'),
 (51, 38, 'B'),
 (51, 38, 'C'),
 (51, 40, 'A'),
 (51, 40, 'B'),
 (51, 40, 'C'),
 (51, 41, 'A'),
 (51, 41, 'B'),
 (51, 41, 'C'),
 (52, 3

In [5]:
def storage_opt(signal, dbcon=None):
    action = [s for s in signal['Locations'] if s['Operation']=='drop'][0]['Name']
    # if signal loc for 'drop' is named 'stage', return signal as-is
    try:
        if action == 'Stage':
            # we will modify this later
            return signal
        elif action == 'Final':
            
            # parse additional data as needed
            data = signal['KPMGAdditionalData']
            # container type
            container_type = data['ContainerType']
            # container priority
            container_priority = data['priority']
            # container dimensions - useful for pallets with unknown heights
            container_height = data['height']
            container_width = data['width']
            container_depth = data['depth']
            container_weught = data['weight']
            
            container_id = signal['LoadIdentifier']
            
            unit = [{'ID': container_id,
                      'Name': container_type,
                      'Length': container_width,
                      'Width': container_depth,
                      'Height': container_height,
                      'Volume': container_width*container_depth*container_height,
                      'Priority': container_priority,
                      'Weight': container_weight}]
            
            '''
            INSERT CODE TO DETERMINE CURRENT STORAGE AVAILABILITY (AZURE SQL)
            '''
    

            if dbcon is None:
                print('Establishing connection to inventory DB')
                dbcon = pyodbc.connect('''Driver={ODBC Driver 18 for SQL Server};Server=tcp:usmc5gsmartwarehousetest.database.windows.net,1433;Database=D365TestDB;Uid=usmc5gadmin;Pwd=Usmc5gp@ssword1;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;''')



            # cursor = connection.cursor()
            query = '''
                SELECT
                s.LICENSEPLATENUMBER AS "LPN",
                l.LICENSEPLATENUMBER AS "LPN2",
                l.CONTAINERTYPEID AS "ContainerType",
                l.PARENTLICENSEPLATENUMBER as "ParentLPN",
                p.CONTAINERTYPEID AS "ParentContainerType",
                s.WAREHOUSELOCATIONID
                FROM "Inventory On-Hand Report Storage" s
                LEFT JOIN "License Plates" l on s.LICENSEPLATENUMBER = l.LICENSEPLATENUMBER
                LEFT JOIN "License Plates" p on l.PARENTLICENSEPLATENUMBER = p.LICENSEPLATENUMBER
                '''
            # cursor.execute(query)
            # column_names = [x[0] for x in cursor.description]
            # extract = cursor.fetchall()
            # df = pd.DataFrame(extract, columns=column_names)

            # connection.close()
            df = pd.read_sql(query, dbcon)

            df = df[df['WAREHOUSELOCATIONID'].notna()]
            df = df[df['WAREHOUSELOCATIONID'].str.match('.2.\d{4}\w{2}')]
            df = df.drop(columns=['LPN2']) # duplicate
            # if an item has a parent lpn, use that instead
            df.loc[df['ParentLPN'].notna(), 'LPN'] = df['ParentLPN']
            # if an item has a parent lpn, use that container type instead
            df.loc[df['ParentContainerType'].notna(), 'ContainerType'] = df['ParentContainerType']
            # drop parent cols now
            df = df.drop(columns=['ParentLPN', 'ParentContainerType'])
            # drop duplicates created from multiple rows having the same parent info
            df = df[~df.duplicated()]
            
            container_ids = [
                'MODULAR NESTING CONT',
                'BULK CONT LENGTHDOOR',
                'BULK CONT WIDTHDOOR',
                'WOOD SHIPPING PALLET'
            ]

            # df
            def loc_id_to_tuple(locid):
                l = locid
                return (int(l[3:5]), int(l[5:7]),l[7])

            mod_nest_counter = {loc_id_to_tuple(locid):0 for locid in df.WAREHOUSELOCATIONID}
            MOD_NEST_MAX = 3
            used_shelf_ids = []
            for idx, row in df.iterrows():
                 # df
                locid = loc_id_to_tuple(row.WAREHOUSELOCATIONID)
                
                if row.ContainerType == 'MODULAR NESTING CONT':
                    mod_nest_counter[locid]+=1
                elif row.ContainerType in ['BULK CONT LENGTHDOOR','BULK CONT WIDTHDOOR','WOOD SHIPPING PALLET']:
                    used_shelf_ids.append(locid)
                    
            for locid, num_mod_nest in mod_nest_counter:
                if container_type == 'MODULAR NESTING CONT':
                    if num_mod_nest > MOD_NEST_MAX:
                        used_shelf_ids.append(locid)
                else:
                    if num_mod_nest >=1:
                        used_shelf_ids.append(locid)
                    
            
            
                
            
            
            
            '''
            READ TRANSACTION HISTORY + UPDATE STORAGE AVAILABILITY
            '''
            # get relevant transaction history data
            connection = sqlite3.connect(db_path)
            cursor = connection.cursor()
            query = '''SELECT * FROM TransactionHistory'''
            cursor.execute(query)
            column_names = [x[0] for x in cursor.description]
            extract = cursor.fetchall()
            df = pd.DataFrame(extract, columns=column_names)
            connection.close()
            
            
            # update storage availability with any updates that we've processed in the last 30 min
            df['AssignmentTime'] = pd.to_datetime(df['AssignmentTime'])
            
            
            '''
            INSERT STORAGE OPTIMIZATION CODE WHERE OUTPUT == final_location
            '''
            
            new = put_away(units=unit,
                               used_shelf_ids=used_shelf_ids)
            new = list(new.keys())[0]
            
            def to_loc_fmt(key_fmt):
                return 'I21'+''.join(list(key_fmt))+'A'
            
            
            final_location = to_loc_fmt(new) # just a sample output that storage may have, but output must be in this format
            
            
            '''
            WRITE DECISION TO TRANSACTION HISTORY
            '''
            connection = sqlite3.connect(db_path)
            cursor = connection.cursor()
            query = '''
                INSERT OR REPLACE INTO TransactionHistory ("LoadIdentifier", "AssignmentTime", "FinalLoc")
                VALUES (?, ?, ?)
            '''
            cursor.execute(query, [signal['LoadIdentifier'], datetime.now().strftime('%Y-%m-%d %H:%M:%S'), final_location])
            connection.commit()
            connection.close()
                    
            # drop KPMGAdditionalData from output - TMO doesn't need it
            signal.pop('KPMGAdditionalData', None)
                    
            return signal 
        else:
            raise Exception('Input data not "final" or "stage".')
    except:
        raise Exception('Error parsing input data.')

In [2]:
import pyodbc
import pandas as pd

con = pyodbc.connect('''Driver={ODBC Driver 18 for SQL Server};Server=tcp:usmc5gsmartwarehousetest.database.windows.net,1433;Database=D365TestDB;Uid=usmc5gadmin;Pwd=Usmc5gp@ssword1;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;''')

query = '''select x.LICENSEPLATENUMBER,x.WAREHOUSELOCATIONID,y.CONTAINERTYPEID 
from "Inventory on-hand report storage" x
left join "License plates" y on x.LICENSEPLATENUMBER=y.LICENSEPLATENUMBER;'''

df = pd.read_sql(query, con)

re = '.2.\d{4}\w{2}'

df

OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 18 for SQL Server]TCP Provider: Timeout error [258].  (258) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Invalid connection string attribute (0); [08001] [Microsoft][ODBC Driver 18 for SQL Server]Unable to complete login process due to delay in login response (258)')

In [9]:
import pyodbc

connection = pyodbc.connect('''Driver={ODBC Driver 18 for SQL Server};Server=tcp:usmc5gsmartwarehousetest.database.windows.net,1433;Database=D365TestDB;Uid=usmc5gadmin;Pwd=Usmc5gp@ssword1;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;''')


# cursor = connection.cursor()
query = '''
    SELECT
    s.LICENSEPLATENUMBER AS "LPN",
    l.LICENSEPLATENUMBER AS "LPN2",
    l.CONTAINERTYPEID AS "ContainerType",
    l.PARENTLICENSEPLATENUMBER as "ParentLPN",
    p.CONTAINERTYPEID AS "ParentContainerType",
    s.WAREHOUSELOCATIONID
    FROM "Inventory On-Hand Report Storage" s
    LEFT JOIN "License Plates" l on s.LICENSEPLATENUMBER = l.LICENSEPLATENUMBER
    LEFT JOIN "License Plates" p on l.PARENTLICENSEPLATENUMBER = p.LICENSEPLATENUMBER
    '''
# cursor.execute(query)
# column_names = [x[0] for x in cursor.description]
# extract = cursor.fetchall()
# df = pd.DataFrame(extract, columns=column_names)

# connection.close()
df = pd.read_sql(query, connection)

df = df[df['WAREHOUSELOCATIONID'].notna()]
df = df[df['WAREHOUSELOCATIONID'].str.match('.2.\d{4}\w{2}')]
df = df.drop(columns=['LPN2']) # duplicate
# if an item has a parent lpn, use that instead
df.loc[df['ParentLPN'].notna(), 'LPN'] = df['ParentLPN']
# if an item has a parent lpn, use that container type instead
df.loc[df['ParentContainerType'].notna(), 'ContainerType'] = df['ParentContainerType']
# drop parent cols now
df = df.drop(columns=['ParentLPN', 'ParentContainerType'])
# drop duplicates created from multiple rows having the same parent info
df = df[~df.duplicated()]
df

,LPN,ContainerType,WAREHOUSELOCATIONID
17,100000000004,None,M212953AA
18,100000000005,None,M212953AA
19,100000000006,None,M212953AA
25,100000000012,None,M212953AA
26,100000000014,None,M213874AA
...,...,...,...
5641,None,None,I210351AA
5642,None,None,I210379AA
5643,None,None,I210380AA
5644,None,None,I214088DA


In [11]:
 # df
def loc_id_to_tuple(locid):
    l = locid
    return (int(l[3:5]), int(l[5:7]),l[7])

used_location_ids = []
for idx, row in df.iterrows():
    used_location_ids.append(loc_id_to_tuple(row.WAREHOUSELOCATIONID))

In [12]:
used_location_ids

[(29, 53, 'A'),
 (29, 53, 'A'),
 (29, 53, 'A'),
 (29, 53, 'A'),
 (38, 74, 'A'),
 (29, 53, 'A'),
 (29, 54, 'B'),
 (29, 53, 'A'),
 (29, 53, 'A'),
 (28, 54, 'B'),
 (29, 54, 'B'),
 (27, 53, 'A'),
 (19, 54, 'B'),
 (29, 53, 'A'),
 (35, 68, 'A'),
 (28, 53, 'B'),
 (29, 54, 'C'),
 (29, 53, 'B'),
 (39, 68, 'C'),
 (82, 5, 'A'),
 (33, 85, 'B'),
 (27, 75, 'A'),
 (27, 75, 'A'),
 (30, 55, 'C'),
 (36, 68, 'C'),
 (26, 65, 'A'),
 (15, 64, 'B'),
 (15, 64, 'B'),
 (27, 75, 'B'),
 (27, 75, 'A'),
 (27, 75, 'A'),
 (27, 75, 'A'),
 (27, 75, 'A'),
 (21, 67, 'A'),
 (21, 67, 'A'),
 (34, 57, 'B'),
 (34, 57, 'B'),
 (27, 75, 'A'),
 (27, 75, 'A'),
 (27, 75, 'A'),
 (27, 75, 'A'),
 (21, 67, 'A'),
 (37, 56, 'B'),
 (16, 65, 'B'),
 (24, 54, 'B'),
 (40, 56, 'B'),
 (34, 57, 'B'),
 (20, 65, 'A'),
 (20, 65, 'A'),
 (23, 65, 'A'),
 (36, 56, 'A'),
 (27, 54, 'B'),
 (39, 74, 'C'),
 (15, 64, 'B'),
 (36, 68, 'C'),
 (20, 65, 'A'),
 (36, 88, 'D'),
 (36, 68, 'B'),
 (36, 68, 'C'),
 (36, 68, 'B'),
 (34, 65, 'A'),
 (40, 72, 'D'),
 (15, 64,

In [7]:
input_path = 'storage_opt_d365_input.json' # modify this based on your machine

with open(input_path) as file:
    signal = json.loads(file.read())
    
storage_opt(signal)

{'TransactionId': 'EVNT000000056',
 'Type': 'pick drop',
 'VehicleID': 0,
 'VehicleType': 'amr',
 'Priority': 0,
 'LoadContainer': '',
 'LoadContent': 'content',
 'LoadIdentifier': 'CONT-000000047',
 'Locations': [{'Operation': 'pick',
   'Name': 'CONV-SP-1',
   'Level': 0,
   'Inventory': 'System'},
  {'Operation': 'drop', 'Name': 'Final', 'Level': 0, 'Inventory': 'System'}]}

In [5]:
# preview db
connection = sqlite3.connect(db_path)
cursor = connection.cursor()
query = '''SELECT * FROM TransactionHistory'''
cursor.execute(query)
column_names = [x[0] for x in cursor.description]
extract = cursor.fetchall()
df = pd.DataFrame(extract, columns=column_names)
connection.close()
df

,LoadIdentifier,AssignmentTime,StagingLoc,FinalLoc
0,CONT-000000047,2022-08-26 15:07:24,None,I213365AA


### Future Directions

In [ ]:
# add stackability for modular nesting containers

In [ ]:
# testing --
# simulate random v. intentional putaway and calculate shelf waste